# 게임회사 다음 분기 계획 결정

## EDA

### 데이터 불러오기
* Name : 게임의 이름입니다.  
* Platform : 게임이 지원되는 플랫폼의 이름입니다.  
* Year : 게임이 출시된 연도입니다.  
* Genre : 게임의 장르입니다.  
* Publisher : 게임을 제작한 회사입니다.  
* NA_Sales : 북미지역에서의 출고량입니다.(단위 100만)  
* EU_Sales : 유럽지역에서의 출고량입니다.(단위 100만)  
* JP_Sales : 일본지역에서의 출고량입니다.(단위 100만)  
* Other_Sales : 기타지역에서의 출고량입니다.(단위 100만)  

In [80]:
import pandas as pd
import numpy as np

In [81]:
# url
data_url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv'
# read
df = pd.read_csv(data_url, index_col = 0)

In [82]:
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [83]:
np.shape(df)

(16598, 9)

### 데이터 중복행 제거

In [84]:
df[df.duplicated()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
9185,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0,0,0.02,0


In [85]:
df = df.drop_duplicates(ignore_index = True)

### 결측치 제거

In [86]:
df.isnull().sum()

Name             0
Platform         0
Year           270
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [87]:
df = df.dropna()

In [88]:
df.isnull().sum()

Name           0
Platform       0
Year           0
Genre          0
Publisher      0
NA_Sales       0
EU_Sales       0
JP_Sales       0
Other_Sales    0
dtype: int64

### 데이터내 이상값, 데이터형 처리
'Name', 'Publisher'는 고유명사이므로 처리하지 않고 코드북이 없을때도 유추할 수 있는 나머지 변수에서 이상값을 찾아 처리를 진행했다.

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16596
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  float64
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  object 
 6   EU_Sales     16241 non-null  object 
 7   JP_Sales     16241 non-null  object 
 8   Other_Sales  16241 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.2+ MB


#### Name

In [129]:
df.loc[:,'Name'].unique()

array(["Candace Kane's Candy Factory", 'The Munchables',
       'Otome wa Oanesama Boku ni Koi Shiteru Portable', ...,
       'White Album: Tsuzurareru Fuyu no Omoide', 'Minna no Chizu 2',
       'Rainbow Islands: Revolution'], dtype=object)

#### Platform
이상값은 없어 보인다

In [90]:
df.loc[:,'Platform'].unique()

array(['DS', 'Wii', 'PSP', 'PS3', 'PC', 'PS', 'GBA', 'PS4', 'PS2', 'XB',
       'X360', 'GC', '3DS', '2600', 'SAT', 'GB', 'NES', 'DC', 'N64',
       'XOne', 'SNES', 'WiiU', 'PSV', 'GEN', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

#### Year
데이터형 변경: 'float' -> 'int'  
* 년도는 정수이므로 데이터형을 변경한다.

70이상 100이하는 1900을 더하고 70미만은 2000을 더함
* 상업적 게임이 나온 첫 시기는 70년대 이므로 70이상은 1900년대로 처리, 70미만은 2000년대로 처리한다.

In [91]:
df.loc[:,'Year'] = df.loc[:,'Year'].astype('int')

In [92]:
df.loc[:,'Year'].unique()

array([2008, 2009, 2010, 2005, 2011, 2007, 2001, 2003, 2006, 2014, 2015,
       2002, 1997, 2013, 1996, 2004, 2000, 1984, 1998, 2016, 1985, 1999,
          9,   97, 1995, 1993, 2012, 1987, 1982,   11, 1994, 1990,   15,
       1992, 1991, 1983, 1988, 1981,    3, 1989,   96,    6,    8, 1986,
          1,    5,    4,   10,   98,    7,   16,   86,   14,   95, 2017,
       1980, 2020,    2,   13,    0,   12,   94])

In [121]:
# 70이상 100이하는 1900을 더하고 70미만은 2000을 더함
df['Year'] = df['Year'].apply(lambda x: x+1900 if (x>=70)&(x<100) else x+2000 if x<70 else x)

In [122]:
df.loc[:,'Year'].unique()

array([2008, 2009, 2010, 2005, 2011, 2007, 2001, 2003, 2006, 2014, 2015,
       2002, 1997, 2013, 1996, 2004, 2000, 1984, 1998, 2016, 1985, 1999,
       1995, 1993, 2012, 1987, 1982, 1994, 1990, 1992, 1991, 1983, 1988,
       1981, 1989, 1986, 2017, 1980, 2020])

#### Genre
이상값은 없어 보인다

In [124]:
df.loc[:,'Genre'].unique()

array(['Action', 'Adventure', 'Misc', 'Platform', 'Sports', 'Simulation',
       'Racing', 'Role-Playing', 'Puzzle', 'Strategy', 'Fighting',
       'Shooter'], dtype=object)

#### Publisher

In [128]:
df.loc[:,'Publisher'].unique()

array(['Destineer', 'Namco Bandai Games', 'Alchemist', 'Zoo Games',
       'D3Publisher', 'Konami Digital Entertainment', 'Deep Silver',
       'Tecmo Koei', 'Sega', 'THQ', 'Empire Interactive', 'Ubisoft',
       'Paradox Interactive', '505 Games', 'Tomy Corporation',
       'Eidos Interactive', 'Microsoft Game Studios',
       'Nippon Ichi Software', 'Kalypso Media', 'Electronic Arts',
       'Sony Computer Entertainment', 'Activision',
       'Tru Blu Entertainment', 'Warner Bros. Interactive Entertainment',
       'Midway Games', 'Mindscape', 'Wargaming.net', 'Capcom',
       'Acclaim Entertainment', 'Unknown', 'Nintendo',
       'Disney Interactive Studios', 'Crave Entertainment', 'Sunflowers',
       'Arc System Works', 'Milestone S.r.l', 'Kamui', 'Hudson Soft',
       'Square Enix', 'Focus Home Interactive', 'Take-Two Interactive',
       'Game Factory', 'Takara Tomy', 'SquareSoft', 'Global Star',
       'Infogrames', 'GT Interactive', 'Storm City Games',
       'Gathering of Dev

#### NA_Sales
숫자, 숫자M, 숫자K 3경우 데이터가 보인다. 이들 단위를 100만으로 맞춰준다.
* 숫자: 단위 100만으로 나둔다.
* 숫자M: 단위는 100만으로 맞으므로 M만을 제거한다.
* 숫자K: 단위가 1천으로 K를 제거 후 1,000으로 나눈다.
* 소수점 3째자리 부터 절삭한 것으로 보인다.  

데이터형 변경: 'object' -> 'float'  
* sales는 출고량이므로 float로 변경(단위 100만)

In [127]:
df.loc[:,'NA_Sales'].unique()

array(['0.04', '0.17', '0', '0.12', '0.02', '0.07', '480K', '0.14',
       '0.01', '0.57', '0.06', '0.08', '0.48', '0.03', '0.05', '0.29',
       '0.19', '0.3', '0.26', '0.11', '0.18', '0.33', '0.51', '60K',
       '0.13', '0.34', '0.25', '0.41', '0.52', '0.22', '0.35', '0.1',
       '0.67', '0.59', '0.36', '0.21', '1.19', '5.91', '1.98', '0.09',
       '0K', '0.5', '0.94', '0.16', '0.63', '0.43', '0.2', '1.23', '0.44',
       '0.42', '0.97', '2.67', '0.28', '6.05', '0.47', '0.49', '0.62',
       '0.24', '0.72', '1.74', '0.46', '0.65', '0.32', '1.53', '0.23',
       '0.53', '0.37', '1.26', '0.39', '0.58', '1.94', '2.31', '1.03',
       '0.82', '0.4', '0.93', '0.56', '0.96', '1.64', '0.69', '1.25',
       '0.27', '0.92', '0.58M', '1.24', '0.25M', '0.86', '0.73', '0.55',
       '1.54', '1.02', '1.27', '0.54', '0.15', '1.09', '3.44', '2.29',
       '0.88', '5.55M', '1.37', '2.3', '0.99', '1.15', '0.98', '0.75',
       '0.74', '0.7', '0.14M', '0.45', '1.2', '1.41', '0.68', '1.23M',
       

지역에 따라서 선호하는 게임 장르가 다를까

연도별 게임의 트렌드가 있을까

출고량이 높은 게임에 대한 분석 및 시각화 프로세스